# OPPT Tutorial (USING)

### OPPT Examples
Tiger Problem Example will be uploaded to a public repository.
I will send the link to the public repository once uploaded.
Feel free to suggest any changes.


### OPPT Documents
Compile an OPPT FAQ document with common difficulties from install, running an example provided, and implementing own problem.
This would help improve OPPT and documentation for future updates.


### Collaboration
Work together in an oppt slack or teams to help each other on common problems faced.

# OPPT handy hacks for testing

#### "getchar()" after each execution step in ProblemEnvironment.hpp
We can place a "getchar()" statement after each execution step to track what is happening during an experiment run: 


""" Example of run function in ProblemEnvironment.hpp """
 SimulationResult run(const unsigned int& run, std::ofstream & os, int argc, char const * argv[]) {
        
        ... Some setup code happening here ...
        
        # Sample an initial state
        RobotStateSharedPtr currentState =
            robotExecutionEnvironment_->sampleInitialState();
       
        ... Some more setup code here ....
        
        FloatType t0 = oppt::clock_ms();
        while (true) {
            cout << "\nt = " << currentStep << endl;
            stepResult = StepResult(currentStep, robotExecutionEnvironment_.get());
            stepResult.currentState = currentState;
            stepResult.discountFactor = currentDiscount;
            if (currentStep >= problemEnvironmentOptions_->nSimulationSteps) {
                // We're out of steps
                PRINT("Out of steps. Ending simulation run");
                finalState = currentState;
                break;
            }

            if (problemEnvironmentOptions_->hasVerboseOutput)
                cout << "Improving policy...\n";

            // Improve the current policy
            FloatType planningTimer = oppt::clock_ms();
            if (!solver_->improvePolicy(problemEnvironmentOptions_->stepTimeout)) {
                finalState = currentState;
                totalPlanningTime += (oppt::clock_ms() - planningTimer) / 1000.0;
                LOGGING("Couldn't improve policy. Ending simulation run");
                break;
            }

            totalPlanningTime += (oppt::clock_ms() - planningTimer) / 1000.0;

            // Get nextAction
            ActionSharedPtr action = solver_->getNextAction();
            if (!action) {
                // No action to execute
                finalState = currentState;
                LOGGING("No action for the next step. Ending simulation run");
                break;
            }

            cout << "action: " <<
                 *(robotPlanningEnvironment_->getRobot()->getActionSpace()->denormalizeAction(action))
                 << endl;
            stepResult.action = action;

            // Execute action
            PropagationRequestSharedPtr propagationRequest(new PropagationRequest());
            propagationRequest->currentState = currentState;
            propagationRequest->action = action.get();
            propagationRequest->allowCollisions = problemEnvironmentOptions_->allowCollisions;

            oppt::PropagationResultSharedPtr propagationResult =
                robotExecutionEnvironment_->getRobot()->propagateState(propagationRequest);
            if (propagationResult->nextState) {
                cout << "Next state: " <<
                     *(robotPlanningEnvironment_->getRobot()->getStateSpace()->denormalizeState(propagationResult->nextState).get())
                     << endl;
            } else {
                cout << "Next state: NULL" << endl;
            }

            // Get an observation
            ObservationRequestSharedPtr observationRequest(new ObservationRequest());
            observationRequest->currentState = propagationResult->nextState;
            observationRequest->action = action.get();
            ObservationResultSharedPtr observationResult =
                robotExecutionEnvironment_->getRobot()->makeObservationReport(observationRequest);

            stepResult.observation = observationResult->observation;
            auto observationSpace = robotPlanningEnvironment_->getRobot()->getObservationSpace();
            cout << "Observation: " <<
                 *(observationSpace->denormalizeObservation(stepResult.observation).get()) <<
                 endl;

            // Get a reward
            FloatType reward = robotExecutionEnvironment_->getReward(propagationResult);

            totalDiscountedReward += currentDiscount * reward;
            stepResult.reward = reward;
            cout << "Immediate reward: " << reward << endl;

            // Update the true state
            currentState = propagationResult->nextState;
            currentDiscount *= problemEnvironmentOptions_->discountFactor;

            // Check if we're terminal
            bool terminal = robotExecutionEnvironment_->isTerminal(propagationResult);

            if (terminal) {
                PRINT("Terminal state reached!");
                finalState = currentState;
                if (reward == robotExecutionEnvironment_->getRewardPlugin()->getMinMaxReward().second)
                    success = true;
            }

            // Update the belief
            if (!solver_->updateBelief(action, observationResult->observation, terminal)) {
                // Couldn't update the belief
                finalState = currentState;
                serializeLastBelief = false;
                LOGGING("Couldn't update belief. Ending simulation run");
￼

                break;
            }

            if (problemEnvironmentOptions_->hasVerboseOutput)
                PRINT("Updated belief");

            beliefParticles = solver_->getBeliefParticles();

            // Update the viewer
            if (viewer && viewer->viewerRunning())
                updateViewer(currentState, beliefParticles);


            // Serialization
            ... Some code for serializing steps ...

            // Update step
            currentStep++;

            // Input between steps
            **getchar();**

            if (terminal)
                break;
        }

# Tiger Problem Plugins Implementation
**Simple Tiger Problem Example**

Ref: https://h2r.github.io/pomdp-py/html/examples.tiger.html#:~:text=The%20description%20of%20the%20tiger,put%20behind%20the%20other%20one.

## Anatomy of Config File

### General-purpose settings (These keywords are used to control the details on the log files produced)

__Verbose output__

verbose = true     

__Logfile output path__

logPath = /home/jimy/Desktop/testing                      

__Replace log files with same destination__

overwriteExistingLogFiles = false       

__keyword postfix to be added to the log file name__
logFilePostfix = tiger_problem_test   

__Include detailed belief output (individual particles for belief nodes at each step would be logged)__

saveParticles = false                                    


### Defining State, Action, Observation Spaces
All 3 spaces can be automatically produced from information from the configuration file.
However, producing the spaces in this manner has some limitations. 

This section will present different ways of redefining or setting your POMDP spaces.
- Bypass state space
- Custom Action Space definition
- Observation Space from config file

## State space definition

- Issue: Currently Error when trying to "set" state space to be 1-D through additionalDimensions in config:
Workaround is to define custom state space. The states considered for the state space are the states that appear from the
        - Initial Belief Plugin (Sampling an initial state)
        - Transition Plugin (How the states evolve)
    
We can encode that coverage of the state space by defining how the initial state is sampled and how states evolve.
(Show Initial Belief Plugin Implementation)
      

## Problem with CFG Definition of Action Space
With the custom cfg definition of the action space (e.g)

Setting "additionalDimensions = 6" with the appropiate limits:
- Considering each dimension has 2 steps:

We have a total of 64 actions

## Custom Action Space definition

Currently, the action space can be automatically defined in the configuration file.
However, the current use of the action space definition relies on the following s inputs
- Num of dimensions of space:
- actionType = discrete
- numInputStepsActions

The problem with this definition is that customing an action space (with custom action vector represenations is tricky
(E.g Robot Problem had 6 dimensional actions (joint1, joint2, ... joint 6)  Each action moved 6 joints. However, to include additional individual actions (Open Gripper) would have
required an additional dimension (which by default could increase the action space complexity unintendedly)


**Can Customize Action Space by implementing custom class inheriting from ActionSpaceDiscretizer.hpp**

(Look at TigerProblemActionSpaceDiscretizer.hpp for example)

## Observation Space Definition
For the tiger problem, we can define the observation space using the __discrete__ type in the config file